### 전국일방통행도로표준데이터
- 데이터 전처리
    : 시도명, 지정사유, 지정연도, 도로폭, 도로차로수, 보차분리여부

- 결측치 처리(column 개별 기준)

- 오류치 처리 (unit의 차이)

- 이상치 처리 (iqr 사용)

- 전처리 내용 정리

In [2]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

pd_rawdata = pd.read_csv('전국일방통행도로표준데이터.csv', encoding='cp949')
# EDA : Exploratory Data Aanalysis
print(pd_rawdata.columns)

Index(['시도명', '시군구명', '도로명', '지정사유', '지정연도', '도로폭', '도로연장', '도로차로수', '보차분리여부',
       '시작점위도', '시작점경도', '종료점위도', '종료점경도', '도로안내표지일련번호', '관리기관명', '관리기관전화번호',
       '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')


In [3]:
column_selection = ['시도명', '지정사유', '지정연도', '도로폭', '도로차로수', '보차분리여부']
pd_data = pd_rawdata[column_selection]

print(pd_data.columns)
print(pd_data.dtypes)
print(pd_data['보차분리여부'])

Index(['시도명', '지정사유', '지정연도', '도로폭', '도로차로수', '보차분리여부'], dtype='object')
시도명        object
지정사유       object
지정연도      float64
도로폭       float64
도로차로수     float64
보차분리여부     object
dtype: object
0       Y
1       Y
2       Y
3       Y
4       Y
       ..
2154    N
2155    N
2156    N
2157    N
2158    N
Name: 보차분리여부, Length: 2159, dtype: object


In [5]:
# Y or N 값이기 때문에 category로 자료형 변경

pd_data['보차분리여부'] = pd_data['보차분리여부'].astype('category')
print(pd_data.dtypes)

시도명         object
지정사유        object
지정연도       float64
도로폭        float64
도로차로수      float64
보차분리여부    category
dtype: object


C:\Users\ITPS\AppData\Local\Temp\ipykernel_6120\3253091795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['보차분리여부'] = pd_data['보차분리여부'].astype('category')


In [6]:
# 지정연도 2159개의 값 중 1609개의 null값
# 유의미한 데이터로 보기에는 힘듦
# 년도만 표기되고 날짜는 표기되지 않아 오브젝트로 자료형 처리
# 지정연도 결측치 처리 -> 지정연도를 추측할 데이터가 없으므로 '알 수 없음'으로 처리
print(pd_data['지정연도'].isna().value_counts())

pd_data['지정연도'] = pd_data['지정연도'].astype(np.object_)

pd_data['지정연도'].replace(np.NaN, '알 수 없음', inplace=True)

print(pd_data.dtypes)

True     1609
False     550
Name: 지정연도, dtype: int64
시도명         object
지정사유        object
지정연도        object
도로폭        float64
도로차로수      float64
보차분리여부    category
dtype: object


C:\Users\ITPS\AppData\Local\Temp\ipykernel_6120\2873691969.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['지정연도'] = pd_data['지정연도'].astype(np.object_)
C:\Users\ITPS\AppData\Local\Temp\ipykernel_6120\2873691969.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['지정연도'].replace(np.NaN, '알 수 없음', inplace=True)


In [7]:

# 지정사유 결측치 처리 -> 사유를 추측할 데이터가 없으므로 '사유없음'으로 처리
pd_data['지정사유'].replace(np.NaN, '사유없음', inplace=True)

C:\Users\ITPS\AppData\Local\Temp\ipykernel_6120\99041670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['지정사유'].replace(np.NaN, '사유없음', inplace=True)


In [8]:
# 도로차로수 결측치 처리
# 도로폭32m의 도로차로수가 NaN으로 입력되어 있음.
# 가장 가까운값의 도로차로수 값을 얻었을 때, 24m의 도로폭이 4차선의 값을 가짐.
# 8미터의 도로폭 차이를 생각해 봤을때, 최소 5차선 이상의 값을 가진다고 판단하여 5차선으로 수정

df = pd.DataFrame(pd_data)
value = df.loc[df['도로차로수'].isna(), '도로폭'].values
print(value)

filtered_df = df[df['도로폭'] == 32]
values = filtered_df['도로차로수'].values
print(values)

value = df.loc[df['도로폭'].between(24, 40), '도로차로수'].values
print(value)


pd_data['도로차로수'].replace(np.NaN, '5', inplace=True)

[32.]
[nan]
[ 4. nan]


C:\Users\ITPS\AppData\Local\Temp\ipykernel_6120\3365862967.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['도로차로수'].replace(np.NaN, '5', inplace=True)


In [15]:
print(pd_data['도로차로수'].shape)
pd_data['도로차로수'].describe

(2159,)


<bound method NDFrame.describe of 0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2154    1.0
2155    1.0
2156    1.0
2157    1.0
2158    1.0
Name: 도로차로수, Length: 2159, dtype: object>

In [21]:
pd_raw = pd_data

#시도명
print(pd_data['시도명'].unique())
print(pd_data['시도명'].hasnans)

#결측치: 없음

# 오류치: 10 --> '강원도'
# print(pd_raw[pd_raw['시도명']=='10'])
filter1 = pd_raw['시도명']=='10'
# pd_data[filter1]['시도명'] = '강원도' #warning
pd_data.loc[filter1, '시도명'] = '강원도'

print(pd_data['시도명'].unique())
print(pd_data['시도명'].hasnans)

['경기도' '경상북도' '제주특별자치도' '인천광역시' '강원도' '경상남도' '서울특별시' '충청남도' '전라남도' '전라북도'
 '광주광역시' '충청북도']
False
['경기도' '경상북도' '제주특별자치도' '인천광역시' '강원도' '경상남도' '서울특별시' '충청남도' '전라남도' '전라북도'
 '광주광역시' '충청북도']
False


In [26]:
# 지정사유

print(pd_data['지정사유'].unique())
print(pd_data['지정사유'].hasnans)


#결측치
na_filter = pd_data['지정사유'].isna()
#print(na_filter.value_counts())
pd_data.loc[na_filter, '지정사유'] = '불분명'   # pd_data['지정사유'].fillna('불분명')

#오류치: 같은내용 다른이름 정리
def e1(x):
    if '원활' in x:
        return '원활'
    elif '불편' in x:
        return '불편'
    elif '안전' in x:
        return '안전'
    elif '혼잡' in x:
        return '혼잡'
    elif '혼자' in x:
        return '혼잡'
    else:
        return x

pd_data.loc[:, '지정사유'] = pd_data.loc[:, '지정사유'].apply(e1)

# 이상치(outline): 없음

print(pd_data['지정사유'].unique())
print(pd_data['지정사유'].hasnans)

['사유없음' '원활' '불편' '안전' '주민건의' '교행불가' '교통소통' '혼잡' '본도로 진입불가' '주택밀집지역'
 '고가도로' '교통사고 다발' '원통전통시장주차장 출입구' '부채도로' '주민편의' '순환차로' '도로협소' '혼자지역'
 '차량교행']
False
['사유없음' '원활' '불편' '안전' '주민건의' '교행불가' '교통소통' '혼잡' '본도로 진입불가' '주택밀집지역'
 '고가도로' '교통사고 다발' '원통전통시장주차장 출입구' '부채도로' '주민편의' '순환차로' '도로협소' '차량교행']
False


C:\Users\ITPS\AppData\Local\Temp\ipykernel_6120\2538289331.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data.loc[:, '지정사유'] = pd_data.loc[:, '지정사유'].apply(e1)


In [28]:
# 지정연도
print(pd_data['지정연도'].unique())
print(pd_data['지정연도'].hasnans)

# 결측치
na_filter = pd_data['지정연도'].isna()
print(na_filter.value_counts())


['알 수 없음' 2017.0 2018.0 2014.0 2005.0 2019.0 2022.0 2016.0 2012.0 1999.0
 2021.0 2015.0 2013.0 2004.0 2002.0 2008.0 2009.0 2010.0 2011.0 1998.0
 2001.0 2003.0 2006.0 2000.0 1995.0 2020.0 2007.0 1990.0 1997.0 1996.0]
False
False    2159
Name: 지정연도, dtype: int64


In [39]:
#도로폭
print(pd_data['도로폭'].unique())
print(pd_data['도로폭'].hasnans)

# 결측치, 오류치 : 없음

# 이상치 : upper보다 큰 겂을 np.NaN으로 교체, 후에 dropna() 이용 샘플 삭제
# print(pd_data['도로폭'].describe())
# print(pd_data['도로폭'].describe())
q1, q3 = pd_data['도로폭'].quantile([0.25, 0.75])
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr
print(lower, upper)
filter1 = pd_data['도로폭']>upper
print(filter1.value_counts())
pd_data.loc[filter1, '도로폭'] = np.NaN

print(pd_data['도로폭'].unique())
print(pd_data['도로폭'].hasnans)

[ 5.   9.   4.   6.   7.5  8.   7.  12.  15.  16.   3.5  5.5  3.  10.
 14.   4.5  6.5 15.5  2.5  7.1  9.8  9.7  6.3  3.7  8.5  8.2 14.6  2.
  4.9  4.6  2.8  4.8  5.6  8.1  6.4  4.1  6.6  5.7  4.7  4.3  3.8  9.1
  7.4  5.3  5.8  4.2 17.  13.5 24.  11.   9.5 22.  19.   3.2  4.4  5.1
  5.9  3.9 32.   5.4  3.4 18.   7.3  3.3 13.   7.6  8.4  5.2  7.8  6.2
  6.7  8.8  7.7  8.3  6.1  6.9  6.8]
False
-0.5 11.5
False    2104
True       55
Name: 도로폭, dtype: int64
[ 5.   9.   4.   6.   7.5  8.   7.   nan  3.5  5.5  3.  10.   4.5  6.5
  2.5  7.1  9.8  9.7  6.3  3.7  8.5  8.2  2.   4.9  4.6  2.8  4.8  5.6
  8.1  6.4  4.1  6.6  5.7  4.7  4.3  3.8  9.1  7.4  5.3  5.8  4.2 11.
  9.5  3.2  4.4  5.1  5.9  3.9  5.4  3.4  7.3  3.3  7.6  8.4  5.2  7.8
  6.2  6.7  8.8  7.7  8.3  6.1  6.9  6.8]
True


In [34]:
# 도로 차로수
print(pd_data['도로차로수'].unique())
print(pd_data['도로차로수'].hasnans)

#결측치: 1개 샘플, 후에 dropna()이용 제서.
na_filter = pd_data['도로차로수'].isna()
print(na_filter.value_counts())

# 오류치: 60
filter1 = pd_data['도로차로수']==60
print(filter1.value_counts())
pd_data.loc[filter1, '도로차로수'] = np.NaN

# 이상치: 없음
print(pd_data['도로차로수'].unique())
print(pd_data['도로차로수'].hasnans)

[1.0 2.0 4.0 3.0 5.0 '5' 60.0]
False
False    2159
Name: 도로차로수, dtype: int64
False    2158
True        1
Name: 도로차로수, dtype: int64
[1.0 2.0 4.0 3.0 5.0 '5' nan]
True


In [41]:
# 보차분리여부
print(pd_data['보차분리여부'].unique())
print(pd_data['보차분리여부'].hasnans)

#결측치: 1개 ' ' 결측치 --> np.NaN로 대체, 후에 dropna()이용 삭제.
na_filter = pd_data['보차분리여부'].isna()

filter1 = pd_data['보차분리여부']==' '
print(filter1.value_counts())
pd_data.loc[filter1, '보차분리여부'] = np.NaN

print(pd_data['보차분리여부'].unique())
print(pd_data['보차분리여부'].hasnans)

['Y', 'N', NaN]
Categories (3, object): [' ', 'N', 'Y']
True
False    2159
Name: 보차분리여부, dtype: int64
['Y', 'N', NaN]
Categories (3, object): [' ', 'N', 'Y']
True


In [10]:
print(pd_data.shape)
print(pd_data.describe(include='all'))

for e in pd_data:
    print(e, pd_data[e].hasnans)
    if pd_data[e].hasnans==True:
        print(pd_data[e].isna().value_counts())

(2159, 6)
          시도명  지정사유    지정연도          도로폭   도로차로수 보차분리여부
count    2159  2159    2159  2159.000000  2159.0   2159
unique     13    32      30          NaN     7.0      3
top     서울특별시  사유없음  알 수 없음          NaN     1.0      N
freq      755  1567    1609          NaN  2042.0   1657
mean      NaN   NaN     NaN     5.917508     NaN    NaN
std       NaN   NaN     NaN     2.297448     NaN    NaN
min       NaN   NaN     NaN     2.000000     NaN    NaN
25%       NaN   NaN     NaN     4.000000     NaN    NaN
50%       NaN   NaN     NaN     6.000000     NaN    NaN
75%       NaN   NaN     NaN     7.000000     NaN    NaN
max       NaN   NaN     NaN    32.000000     NaN    NaN
시도명 False
지정사유 False
지정연도 False
도로폭 False
도로차로수 False
보차분리여부 False
